In [14]:
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary
from gensim.models import LdaModel
import pandas as pd
import gensim
from gensim import corpora
from nltk.corpus import stopwords
import spacy


data = pd.read_csv("data.csv")
print(data['content'].info())
dataFiltered = data["content"][:35000]

stop_words = stopwords.words('english')

# function to remove stopwords
def remove_stopwords(text):
    textArr = text.split(' ')
    rem_text = " ".join([i for i in textArr if i not in stop_words])
    return rem_text

# remove stopwords from the text
dataFiltered=dataFiltered.apply(remove_stopwords)

# nlp = en_core_web_sm.load()
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def lemmatization(texts,allowed_postags=['NOUN', 'ADJ']): 
       output = []
       for sent in texts:
             doc = nlp(sent) 
             output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags ])
       return output

text_list=dataFiltered.tolist()
#print(text_list[5])
tokenized_reviews = lemmatization(text_list)
#print(tokenized_reviews[5])

# -- Omar was here --------------------------------------------------------------------------------

dictionary = corpora.Dictionary(tokenized_reviews)

doc_term_matrix = [dictionary.doc2bow(doc) for doc in tokenized_reviews]

# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(corpus=doc_term_matrix, id2word=dictionary, num_topics=20, random_state=100, chunksize=100, passes=10)
print("-"*10, "\n", "Topics\n", ldamodel.print_topics(num_words=10))



<class 'pandas.core.series.Series'>
RangeIndex: 50000 entries, 0 to 49999
Series name: content
Non-Null Count  Dtype 
--------------  ----- 
50000 non-null  object
dtypes: object(1)
memory usage: 390.8+ KB
None


/home/shehab/.local/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/home/shehab/.local/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/home/shehab/.local/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/home/shehab/.local/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/home/shehab/.local/lib/python3.10/site-packages/catalog

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, doc_term_matrix, dictionary)
vis


/home/shehab/.local/lib/python3.10/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)
/home/shehab/.local/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/home/shehab/.local/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/home/shehab/.local/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's do

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.138639  0.042488       1        1  19.081556
16     0.158459  0.034626       2        1  10.605799
2      0.131522  0.051798       3        1   7.424049
4      0.025004  0.118566       4        1   6.629095
13     0.183327 -0.098729       5        1   5.919838
19     0.142859  0.026335       6        1   5.676671
12     0.130379  0.077756       7        1   5.512668
9      0.172108 -0.098719       8        1   5.419427
7     -0.055433  0.167998       9        1   4.963993
1      0.057578  0.234823      10        1   4.512274
10    -0.100457  0.084831      11        1   3.475968
8      0.034392 -0.193551      12        1   3.468234
17    -0.028350 -0.159540      13        1   3.284219
6     -0.009987 -0.224843      14        1   3.220693
5     -0.062236  0.058671      15        1   2.761581
18    -0.100846  0.016564      16        1   2.200371
11    -0.179170  0.039935      17        1   1.854946
14    -0.040885 -0.154021      18        1   1.714565
0     -0.271260  0.041927      19        1   1.595043
15    -0.325644 -0.066914      20        1   0.679009, topic_info=          Term          Freq         Total Category  logprob  loglift
696      woman  28435.000000  28435.000000  Default  30.0000  30.0000
493    percent  25149.000000  25149.000000  Default  29.0000  29.0000
503     police  14796.000000  14796.000000  Default  28.0000  28.0000
675      video  12781.000000  12781.000000  Default  27.0000  27.0000
219   campaign  24287.000000  24287.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
704         ’s     77.891630  10097.439044  Topic20  -6.1034   0.1276
2120       com     31.962867   3797.045439  Topic20  -6.9941   0.2149
80        late     15.286399   7356.127411  Topic20  -7.7317  -1.1840
829       good     21.411222  17662.839388  Topic20  -7.3948  -1.7230
3002      pick      1.916726    589.863336  Topic20  -9.8081  -0.7370

[1274 rows x 6 columns], token_table=      Topic      Freq Term
term                      
1879      4  0.070729    .
1879      8  0.020406    .
1879     11  0.026804    .
1879     12  0.634659    .
1879     13  0.051707    .
...     ...       ...  ...
704       9  0.005348   ’s
704      10  0.004358   ’s
704      12  0.008121   ’s
704      16  0.008220   ’s
704      20  0.007725   ’s

[3152 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 17, 3, 5, 14, 20, 13, 10, 8, 2, 11, 9, 18, 7, 6, 19, 12, 15, 1, 16])

In [ ]:
print('\nPerplexity: ', ldamodel.log_perplexity(doc_term_matrix,total_docs=10000))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=ldamodel, texts=tokenized_reviews, dictionary=dictionary , coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -10.600741317138842

Coherence Score:  0.5035028231748266
